In [1]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

messages = [
    {
        "role": "system",
        "text": "Ты умный ассистент",
    },
    {
        "role": "user",
        "text": "Привет! Какими науками занимался Альберт Эйнштейн?",
    },
]

def main():
    sdk = YCloudML(
        folder_id="b1g5t9rqg85fvc523apg",
        auth=IAMTokenAuth(''),
        enable_server_data_logging=False,
    )

    result = (
        sdk.models.completions("yandexgpt-lite").configure(temperature=0.6).run(messages)
    )

    for alternative in result:
        print(alternative)

if __name__ == "__main__":
    main()

Alternative(role='assistant', text='Альберт Эйнштейн внёс значительный вклад в развитие физики. Он наиболее известен своими работами в области **теоретической физики**, особенно **теории относительности** и **квантовой теории**.\n\nСреди его научных достижений:\n* создание специальной и общей теорий относительности;\n* разработка квантовой теории фотоэффекта;\n* открытие индуцированного излучения;\n* объяснение броуновского движения.', status=<AlternativeStatus.FINAL: 3>, tool_calls=None)


In [ ]:
Потоковая генерация:

In [2]:
from __future__ import annotations
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

messages = [
    {"role": "system", "text": "Найди ошибки в тексте и исправь их"},
    {"role": "user", "text": "Ашипки саме сибя ни исрпвят."},
]


def main():
    sdk = YCloudML(
        folder_id="b1g5t9rqg85fvc523apg",
        auth=IAMTokenAuth(''),
        enable_server_data_logging=False,
    )

    model = sdk.models.completions("yandexgpt-lite")

    for result in model.configure(temperature=0.5).run_stream(messages):
        for alternative in result:
            print(alternative)


if __name__ == "__main__":
    main()


Alternative(role='assistant', text='О', status=<AlternativeStatus.PARTIAL: 1>, tool_calls=None)
Alternative(role='assistant', text='Ошибки сами себя не исправят.', status=<AlternativeStatus.FINAL: 3>, tool_calls=None)


In [ ]:
Асинхронная генерация:

In [ ]:
from __future__ import annotations
import time
from yandex_cloud_ml_sdk import YCloudML
from yandex_cloud_ml_sdk.auth import IAMTokenAuth

messages_1 = [
    {
        "role": "system",
        "text": "Найди ошибки в тексте и исправь их",
    },
    {
        "role": "user",
        "text": """Ламинат подойдет для укладке на кухне или в детской 
комнате – он не боиться влаги и механических повреждений благодаря 
защитному слою из облицованных меламиновых пленок толщиной 0,2 мм и 
обработанным воском замкам.""",
    },
]

messages_2 = [
    {"role": "system", "text": "Найди ошибки в тексте и исправь их"},
    {"role": "user", "text": "Ашипки саме сибя ни исрпвят."},
]


def main():

    sdk = YCloudML(
        folder_id="b1g5t9rqg85fvc523apg",
        auth=IAMTokenAuth(''),
        enable_server_data_logging=False,
    )

    model = sdk.models.completions("yandexgpt-lite")

    # Variant 1: подождать выполнения операции, используя 5мс периоды time.sleep(5)

    print("Variant 1:")

    operation = model.configure(temperature=0.5).run_deferred(messages_1)

    status = operation.get_status()
    while status.is_running:
        time.sleep(5)
        status = operation.get_status()

    result = operation.get_result()
    print(result)

    # Variant 2: подождать выполнения операции через метод wait()

    print("Variant 2:")

    operation = model.run_deferred(messages_2)

    result = operation.wait()
    print(result)


if __name__ == "__main__":
    main()
